In [1]:
from PIL import Image, ImageFilter
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import matplotlib.pyplot as plt
from pathlib import Path
import random
import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
import cv2

In [2]:
base_path = "data/100"

# Get tumour file paths and shuffle
tumour_files = []
tumour_dirs = [
    "Invasive_Tumor",
    "Prolif_Invasive_Tumor",
    "T_Cell_and_Tumor_Hybrid"
]

for dir_name in tumour_dirs:
    dir_path = os.path.join(base_path, dir_name)
    if os.path.isdir(dir_path):
        files = [os.path.join(dir_path, f) for f in os.listdir(dir_path)]
        tumour_files.extend(files)

random.shuffle(tumour_files)

# Get immune file paths and shuffle
immune_files = []
immune_dirs = [
    "CD4+_T_Cells", "CD4+_T_Cells", 
    "CD8+_T_Cells", 
    "B_Cells", 
    "Mast_Cells", 
    "Macrophages_1", 
    "Macrophages_2", 
    "LAMP3+_DCs",
    "IRF7+_DCs"
]

for dir_name in immune_dirs:
    dir_path = os.path.join(base_path, dir_name)
    if os.path.isdir(dir_path):
        files = [os.path.join(dir_path, f) for f in os.listdir(dir_path)]
        immune_files.extend(files)

random.shuffle(immune_files)


# Get stromal file paths and shuffle
stromal_files = []
stromal_dirs = [
    "Stromal", 
    "Stromal_and_T_Cell_Hybrid", 
    "Perivascular-Like"
]

for dir_name in stromal_dirs:
    dir_path = os.path.join(base_path, dir_name)
    if os.path.isdir(dir_path):
        files = [os.path.join(dir_path, f) for f in os.listdir(dir_path)]
        stromal_files.extend(files)

random.shuffle(stromal_files)

# Get other file paths and shuffle
other_files = []
other_dirs = [
    "Endothelial",
    "Myoepi_ACTA2+", 
    "Myoepi_KRT15+", 
    "DCIS_1", 
    "DCIS_2", 
    "Unlabeled"
]

for dir_name in stromal_dirs:
    dir_path = os.path.join(base_path, dir_name)
    if os.path.isdir(dir_path):
        files = [os.path.join(dir_path, f) for f in os.listdir(dir_path)]
        other_files.extend(files)

random.shuffle(other_files)


In [3]:
def load_resize(img_path, size=(224,224)):
    img = Image.open(img_path).convert('RGB')
    img = img.resize(size)
    return np.array(img)

In [4]:
tumour_imgs = [load_resize(f) for f in tumour_files]
print("tumour loaded")

immune_imgs = [load_resize(f) for f in immune_files]
print("immune loaded")

stromal_imgs = [load_resize(f) for f in stromal_files]
print("stromal loaded")

other_imgs = [load_resize(f) for f in other_files]
print("other loaded")

# Train using 80% of data from each group
'''
tumour_train_ind = int(0.8 * len(tumour_imgs))
tumour_test_ind = int(0.2 * len(tumour_imgs))

immune_train_ind = int(0.8 * len(immune_imgs))
immune_test_ind = int(0.2 * len(immune_imgs))

stromal_train_ind = int(0.8 * len(stromal_imgs))
stromal_test_ind = int(0.2 * len(stromal_imgs))

other_train_ind = int(0.8 * len(other_imgs))
other_test_ind = int(0.2 * len(other_imgs))
'''

tumour_train_ind = 1000
tumour_test_ind = 1200

immune_train_ind = 1000
immune_test_ind = 1200

stromal_train_ind = 1000
stromal_test_ind = 1200

other_train_ind = 1000
other_test_ind = 1200

imgs_train = immune_imgs[:immune_train_ind] + tumour_imgs[:tumour_train_ind] + stromal_imgs[:stromal_train_ind] + other_imgs[:other_train_ind]
imgs_test = immune_imgs[immune_train_ind:immune_test_ind] + tumour_imgs[tumour_train_ind:tumour_test_ind] + stromal_imgs[stromal_train_ind:stromal_test_ind] + other_imgs[other_train_ind:other_test_ind]

Xmat_train = np.stack(imgs_train, axis=0)
Xmat_test = np.stack(imgs_test, axis=0)

y_train = ['Immune'] * immune_train_ind + ['Tumour'] * tumour_train_ind + ['Stromal'] * stromal_train_ind + ['Other'] * other_train_ind
y_test = ['Immune'] * immune_test_ind + ['Tumour'] * tumour_test_ind + ['Stromal'] * stromal_test_ind + ['Other'] * other_test_ind

tumour loaded
immune loaded
stromal loaded
other loaded


In [ ]:
def apply_blur(images):
    return np.array([cv2.GaussianBlur(img, (5, 5), 0) for img in images])

def apply_stretch(images):
    stretched = []
    for img in images:
        h, w, c = img.shape
        new_w = int(w * 1.2)  # stretch width by 20%
        img_stretched = cv2.resize(img, (new_w, h))
        img_cropped = img_stretched[:, :w, :]  # crop back to original width
        stretched.append(img_cropped)
    return np.array(stretched)

def apply_greyscale(images):
    greyscale = []
    for img in images:
        grey = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        grey_3ch = cv2.cvtColor(grey, cv2.COLOR_GRAY2RGB)
        greyscale.append(grey_3ch)
    return np.array(greyscale)

def apply_rotation(images):
    rotated = []
    for img in images:
        (h, w) = img.shape[:2]
        center = (w // 2, h // 2)
        matrix = cv2.getRotationMatrix2D(center, 15, 1.0)  # rotate 15 degrees
        rotated_img = cv2.warpAffine(img, matrix, (w, h), borderMode=cv2.BORDER_REFLECT)
        rotated.append(rotated_img)
    return np.array(rotated)

In [ ]:
Xmat_train_original = Xmat_train
Xmat_train_blur = apply_blur(Xmat_train)
Xmat_train_stretch = apply_stretch(Xmat_train)
Xmat_train_greyscale = apply_greyscale(Xmat_train)
Xmat_train_rotate = apply_rotation(Xmat_train)

In [5]:

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

In [6]:
class NumpyImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        image = Image.fromarray((image * 255).astype('uint8'))  # Convert to PIL Image

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = NumpyImageDataset(Xmat_train_original, y_train_enc, transform=transform)
test_dataset = NumpyImageDataset(Xmat_test, y_test_enc, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=64)

In [8]:
tf.keras.backend.clear_session()
K.clear_session()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)

# Freeze feature extractor (optional)
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for 4 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

model = model.to(device)

/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/jason/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████████████████████████████████| 44.7M/44.7M [00:07<00:00, 6.06MB/s]


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [10]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1, Loss: 1.2882
Epoch 2, Loss: 1.1134
Epoch 3, Loss: 1.0648
Epoch 4, Loss: 1.0473
Epoch 5, Loss: 1.0250


In [26]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to calculate gradients during evaluation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Pick class with highest probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Original Accuracy: {accuracy:.2f}%")

Test Accuracy: 15.00%


In [ ]:
# Test with blur augmentation
train_dataset = NumpyImageDataset(Xmat_train_blur, y_train_enc, transform=transform)

tf.keras.backend.clear_session()
K.clear_session()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)

# Freeze feature extractor (optional)
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for 4 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
# Evaluate Model
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to calculate gradients during evaluation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Pick class with highest probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Blur Accuracy: {accuracy:.2f}%")

In [ ]:
# Test with stretch augmentation
train_dataset = NumpyImageDataset(Xmat_train_stretch, y_train_enc, transform=transform)

tf.keras.backend.clear_session()
K.clear_session()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)

# Freeze feature extractor (optional)
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for 4 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to calculate gradients during evaluation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Pick class with highest probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Stretch Accuracy: {accuracy:.2f}%")

In [ ]:
# Test with greyscale augmentation
train_dataset = NumpyImageDataset(Xmat_train_greyscale, y_train_enc, transform=transform)

tf.keras.backend.clear_session()
K.clear_session()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)

# Freeze feature extractor (optional)
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for 4 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to calculate gradients during evaluation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Pick class with highest probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Greyscale Accuracy: {accuracy:.2f}%")

In [ ]:
# Test with rotate augmentation
train_dataset = NumpyImageDataset(Xmat_train_rotate, y_train_enc, transform=transform)

tf.keras.backend.clear_session()
K.clear_session()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)

# Freeze feature extractor (optional)
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for 4 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to calculate gradients during evaluation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Pick class with highest probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Rotation Accuracy: {accuracy:.2f}%")